In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import ttest_ind
from sqlalchemy import create_engine
import warnings

%matplotlib inline
sns.set()

warnings.filterwarnings('ignore')

In [19]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df_used = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

## Determine all the variable types and find the fraction of the missing values for each variable.

In [20]:
#determine variable types

df_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

In [21]:
#find fraction of null values (as a percentage)

(df_used.isnull().sum()*100) / df_used.isnull().count()

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          17.627346
TOTAL_REVENUE                   14.209115
FEDERAL_REVENUE                 14.209115
STATE_REVENUE                   14.209115
LOCAL_REVENUE                   14.209115
TOTAL_EXPENDITURE               14.209115
INSTRUCTION_EXPENDITURE         14.209115
SUPPORT_SERVICES_EXPENDITURE    14.209115
OTHER_EXPENDITURE               17.627346
CAPITAL_OUTLAY_EXPENDITURE      14.209115
GRADES_PK_G                     11.595174
GRADES_KG_G                      8.847185
GRADES_4_G                       8.780161
GRADES_8_G                       8.780161
GRADES_12_G                      8.780161
GRADES_1_8_G                     8.780161
GRADES_9_12_G                    8.780161
GRADES_ALL_G                    11.595174
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

## Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

In [27]:
df_used2 = df_used.copy()

#choosing all float columns with < 20% missing values for mean imputation
columns = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE", 
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G", 
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G", "GRADES_9_12_G", "GRADES_ALL_G"]

#process mean imputation
for column in columns:
    df_used2.loc[:, column].fillna(df_used2.loc[:, column].mean(), inplace=True)
    
print(df_used2.describe())

              YEAR        ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  \
count  1492.000000  1.492000e+03   1.492000e+03     1.492000e+03   
mean   2004.433646  9.159308e+05   9.092082e+06     7.663723e+05   
std       7.393983  9.667725e+05   1.087818e+07     1.060702e+06   
min    1992.000000  4.386600e+04   4.656500e+05     3.102000e+04   
25%    1998.000000  3.150940e+05   2.546261e+06     2.170200e+05   
50%    2004.000000  8.204140e+05   6.359310e+06     5.167410e+05   
75%    2011.000000  9.211780e+05   9.276958e+06     7.663723e+05   
max    2017.000000  6.307022e+06   8.921726e+07     9.990221e+06   

       STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  \
count   1.492000e+03   1.492000e+03       1.492000e+03   
mean    4.216553e+06   4.109157e+06       9.196681e+06   
std     5.133895e+06   5.078230e+06       1.109393e+07   
min     0.000000e+00   2.209300e+04       4.816650e+05   
25%     1.356434e+06   9.447962e+05       2.523968e+06   
50%     3.127639e+06   2.697257e+06    

## Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

In [30]:
df_used3 = df_used.copy()

years = df_used3["YEAR"].unique()

for column in columns:
    for year in years:
        df_used3.loc[df_used3["YEAR"] == year, col].fillna(
            df_used3[df_used3["YEAR"] == year][col].mean(), inplace=True)
        
print(df_used3.describe())

              YEAR        ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  \
count  1492.000000  1.229000e+03   1.280000e+03     1.280000e+03   
mean   2004.433646  9.159308e+05   9.092082e+06     7.663723e+05   
std       7.393983  1.065280e+06   1.174519e+07     1.145242e+06   
min    1992.000000  4.386600e+04   4.656500e+05     3.102000e+04   
25%    1998.000000  2.583140e+05   2.186305e+06     1.893540e+05   
50%    2004.000000  6.483130e+05   5.079546e+06     4.033765e+05   
75%    2011.000000  1.014528e+06   1.085985e+07     8.289660e+05   
max    2017.000000  6.307022e+06   8.921726e+07     9.990221e+06   

       STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  \
count   1.280000e+03   1.280000e+03       1.280000e+03   
mean    4.216553e+06   4.109157e+06       9.196681e+06   
std     5.543072e+06   5.482971e+06       1.197813e+07   
min     0.000000e+00   2.209300e+04       4.816650e+05   
25%     1.153097e+06   7.158345e+05       2.165404e+06   
50%     2.537074e+06   2.055780e+06    

## This time, fill in the missing values using interpolation (extrapolation).

In [32]:
df_used4 = df_used.copy()

#process interpolation
for column in columns:
    df_used4.loc[:, col].interpolate(inplace=True)
    
print(df_used4.describe())

              YEAR        ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  \
count  1492.000000  1.456000e+03   1.280000e+03     1.280000e+03   
mean   2004.433646  1.005984e+06   9.092082e+06     7.663723e+05   
std       7.393983  1.009158e+06   1.174519e+07     1.145242e+06   
min    1992.000000  4.386600e+04   4.656500e+05     3.102000e+04   
25%    1998.000000  2.973995e+05   2.186305e+06     1.893540e+05   
50%    2004.000000  7.540845e+05   5.079546e+06     4.033765e+05   
75%    2011.000000  1.595024e+06   1.085985e+07     8.289660e+05   
max    2017.000000  6.307022e+06   8.921726e+07     9.990221e+06   

       STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  \
count   1.280000e+03   1.280000e+03       1.280000e+03   
mean    4.216553e+06   4.109157e+06       9.196681e+06   
std     5.543072e+06   5.482971e+06       1.197813e+07   
min     0.000000e+00   2.209300e+04       4.816650e+05   
25%     1.153097e+06   7.158345e+05       2.165404e+06   
50%     2.537074e+06   2.055780e+06    

## Compare your results for the second, third and the fourth questions. Do you find any meaningful differences?

The differences in this case appear to be small, +/- < 10%

In [ ]:
tuesday 7:30 pm